## QA model agent
This Notebook covers the full QA chain. With Self Query retrieving, Memory, Climate Tool, PDF generation, Agent and evaluation.

# Downloads

In [67]:
# !pip install -q "langchain>=0.2.10" langchain-community langchain-text-splitters chromadb
# !pip install -q "sentence-transformers==2.7.0" tiktoken
# !pip install -q langchain-openai openai python-dotenv
# !pip install -q lark
# !pip install -q langsmith
# !pip install -q langchain-community serpapi
# !pip uninstall -y serpapi google-search-results
# !pip install -U google-search-results==2.4.2
# !pip install fpdf2
# !pip install --upgrade fpdf2

In [68]:
# debug/inspect cell
import inspect, serpapi
print("serpapi module path:", serpapi.__file__)
# should be inside .../site-packages/serpapi/ and contain GoogleSearch
from serpapi import GoogleSearch
print("GoogleSearch OK:", inspect.isclass(GoogleSearch))

serpapi module path: c:\Users\Acer\anaconda3\Lib\site-packages\serpapi\__init__.py
GoogleSearch OK: True


# Imports

In [69]:
from pathlib import Path
import os
from typing import List, Dict

# Chroma + embeddings 
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# Self-Query retriever
from langchain.chains.query_constructor.schema import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

# LLM for routing / self-query / QA 
from langchain_openai import ChatOpenAI           

# Compression to trim context
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

# Simple QA chain bits
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

# Session-scoped memory
from collections import defaultdict
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Hardiness zone lookup tool
import os, re, json, time
from typing import Optional
from langchain_core.tools import tool
from langchain_community.utilities import SerpAPIWrapper
from collections import Counter
import requests

# PDF Tool
from fpdf import FPDF
from datetime import datetime

# Agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_openai_tools_agent, AgentExecutor

# Evaluation
from difflib import SequenceMatcher


# Api keys

In [70]:
from dotenv import load_dotenv, find_dotenv
import os
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY') 
LANGSMITH_API_KEY = os.getenv('LANGSMITH_API_KEY') 
SERPAPI_API_KEY = os.getenv('SERPAPI_API_KEY')

## Loading chroma in to notebook

In [71]:
DB_DIR = "./chroma_growguide"
EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"

embeddings = HuggingFaceEmbeddings(
    model_name=EMBED_MODEL,
    model_kwargs={"device": "cpu"}   # stick to CPU unless using have CUDA
)

vectordb = Chroma(
    persist_directory=DB_DIR,
    embedding_function=embeddings
)

print("Docs in DB:", vectordb._collection.count())

Docs in DB: 350


## Self query retrieving
Using LLM (gpt-4o-mini) to retrieve from the vectore storage. By explaining what and where meta data can be found in the data set. If user then uses a term like a month or zone, then this retriever should be able to recognise that and find correct data quickly.

In [72]:
# Self-Query Retriever over Chroma 
# Light LLM to translate questions -> (semantic query + metadata filters)
llm_routing = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Describing metadata so the retriever knows what it can filter on
metadata_field_info = [
    AttributeInfo(
        name="category",
        description="Which section the chunk comes from: 'fruits', 'vegetable_planner', or 'vegetable_list'.",
        type="string",
    ),
    AttributeInfo(
        name="crop",
        description="Vegetable crop name for encyclopedia entries (e.g., onion, tomato, pea).",
        type="string",
    ),
    AttributeInfo(
        name="topics",
        description="Comma-separated keywords inside a vegetable crop entry (yield, planting, conditions, soil, care, companions, harvest, storage tips).",
        type="string",
    ),
    AttributeInfo(
        name="zones",
        description="Planner zone label like '3 and 4', '5 and 6', or '7, 8 and 9+'.",
        type="string",
    ),
    AttributeInfo(
        name="month",
        description="Planner month name when present (e.g., March, April).",
        type="string",
    ),
]

# Brief description of what’s in the documents / dataset
document_contents = (
    "A home-growing guide covering fruits, a vegetable planting/harvest planner by USDA zones and months, "
    "and a per-vegetable encyclopedia with labeled fields such as planting, care, companions, storage, soil."
)

# Building the self-query retriever on top of existing vectordb
self_query_retriever = SelfQueryRetriever.from_llm(
    llm=llm_routing,
    vectorstore=vectordb,
    document_contents=document_contents,
    metadata_field_info=metadata_field_info,
    enable_limit=True,
    search_kwargs={"k": 6},             # k: final number of docs returned
)

print("Self-Query retriever ready.")


Self-Query retriever ready.


## Testing self query retriever 

In [73]:
test_queries = [
    "Which month should I plant carrots?",
    "Companion plants for tomatoes",
    "Best soil pH and watering for fruit trees",
    "How to store onions after harvest?",
]

for q in test_queries:
    print(f"\nQ: {q}")
    hits = self_query_retriever.get_relevant_documents(q)
    for h in hits:
        print(" ->", h.metadata)



Q: Which month should I plant carrots?
 -> {'crop': 'carrots', 'category': 'vegetable_list'}
 -> {'crop': 'carrots', 'category': 'vegetable_list'}
 -> {'crop': 'carrots', 'category': 'vegetable_list'}
 -> {'category': 'vegetable_list', 'crop': 'leek'}
 -> {'crop': 'parsnips', 'category': 'vegetable_list'}
 -> {'crop': 'chicory, belgian endive, french endive, or radicchio', 'category': 'vegetable_list'}

Q: Companion plants for tomatoes
 -> {'category': 'vegetable_list', 'crop': 'cooking, saucing, and paste tomatoes'}
 -> {'crop': 'cherry, grape, or miniature tomatoes', 'category': 'vegetable_list'}
 -> {'crop': 'cherry, grape, or miniature tomatoes', 'category': 'vegetable_list'}
 -> {'crop': 'cooking, saucing, and paste tomatoes', 'category': 'vegetable_list'}
 -> {'category': 'vegetable_list', 'crop': 'slicing or eating tomatoes'}
 -> {'category': 'vegetable_list', 'crop': 'husk tomatoes and ground cherries'}

Q: Best soil pH and watering for fruit trees
 -> {'topic': 'match fruit v

## Building the QA chain
Langsmith has been added to env. file to start tracking the self query retriever, qa chain and later agent and tools.

trying with a basic prompt for now just to see some results.

In [74]:
# helper to show where each snippet came from
def format_docs(docs):
    parts = []
    for d in docs:
        m = d.metadata
        tag = f"[{m.get('category')}"
        if m.get("crop"):   tag += f"|{m['crop']}"
        if m.get("zones"):  tag += f"|zones:{m['zones']}"
        if m.get("month"):  tag += f"|month:{m['month']}"
        tag += "]"
        parts.append(f"{tag} {d.page_content}")
    return "\n\n".join(parts)

# prompt
qa_prompt = PromptTemplate.from_template(
    """You are a helpful home-growing assistant.

SCOPE
• In-scope topics: home fruit & vegetable growing, hardiness climate zones, and growing/seasonal planning (including month-by-month tasks).
• If the QUESTION is outside this scope (e.g., gaming, finance, travel, general tech, etc.) OR the CONTEXT does not relate to those topics, do NOT answer.
  Instead reply something along the lines of:
  "Sorry, this isn’t my expertise. I’m focused on home fruit & vegetable growing, hardiness climate zones, and planning. I can help with things like crops, soil, seasons, zones, or create a growing planner."

RULES
• Use ONLY the provided CONTEXT. If the answer is not in the context, say you don’t know.
• Keep it brief and focused: 1–4 sentences or a short bullet list that directly answers the question.
• Do not add unrelated background or long how-tos unless asked.
• NOT necessary but If you think it is helpful, offer to provided more information, something alone the lines of: “I can share step-by-step details or a month-by-month plan if you’d like.”
• If the CONTEXT includes a crop, month, or zone, tailor the answer to that; otherwise keep it general.
• Do NOT mention “context” or include citations in your answer.

QUESTION: {question}

CONTEXT:
{context}
"""
)

# LLM to generate the answer
llm_answer = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# add LangSmith run names/tags
sq_retriever_traced = self_query_retriever.with_config(
    {"run_name": "self_query_retriever", "tags": ["retriever","self_query","chroma"]}
)

rag_chain = (
    {"context": sq_retriever_traced | format_docs, "question": RunnablePassthrough()}
    | qa_prompt
    | llm_answer
    | StrOutputParser()
).with_config({"run_name": "rag_qa_chain", "tags": ["qa","rag","gpt-4o-mini"]})

print("RAG QA chain ready.")

RAG QA chain ready.


Now testing the qa chain with the test questions

In [75]:
test_questions = [
    "Which month should I plant carrots?",
    "Companion plants for tomatoes",
    "Best soil pH and watering for fruit trees",
    "How to store onions after harvest?",
]

for q in test_questions:
    print("\nQ:", q)
    print(rag_chain.invoke(q))



Q: Which month should I plant carrots?
Carrots are typically planted in early spring or late summer, depending on your climate. In cooler regions, sow seeds about 2-4 weeks before the last expected frost date. In warmer areas, you can also plant in late summer for a fall harvest. If you need a more specific month-by-month plan, I can help with that!

Q: Companion plants for tomatoes
Companion plants for tomatoes include:

- Asparagus
- Basil
- Borage
- Calendula
- Carrots
- Chamomile
- Chives
- Garlic
- Marigolds
- Onions
- Parsley
- Shallots

These plants can help improve growth and deter pests. If you need more information on how to plant or care for these companions, let me know!

Q: Best soil pH and watering for fruit trees
For most fruit trees, the ideal soil pH range is between 6.0 to 7.0, while blueberries specifically require a more acidic pH of 4.5 to 5.5. 

In terms of watering, young fruit trees need regular, deep watering 1–2 times per week for the first 1–2 years, while m

In [76]:
test_questions = [
    "What should i plant in May, i live in zone 6?",
    "What to look outfor when planting rhubarb",
    "What soil conditions do i need with potatoes",
    "What are some easy plants to grow in my garden?",
]

for q in test_questions:
    print("\nQ:", q)
    print(rag_chain.invoke(q))


Q: What should i plant in May, i live in zone 6?
In May, in zone 6, you can plant the following crops:

- Cucumbers (55–65 days)
- Eggplants (100–140 days)
- Endive (85–100 days)
- Garlic (90–100 days)
- Gourds (85–100 days)
- Kale (55–75 days)
- Kohlrabi (45–60 days)
- Lettuce, Head (65–90 days)
- Melons, Summer (70–100 days)
- Okra (55–65 days)
- Peas, Garden (55–70 days)
- Potatoes (90–140 days)
- Pumpkins (90–120 days)

Let me know if you need more details or a month-by-month plan!

Q: What to look outfor when planting rhubarb
When planting rhubarb, consider the following:

- **Timing**: Sow seeds or plant root divisions in early spring when soil temperatures reach 40–85°F (5–29°C) or in autumn in mild-winter climates.
- **Spacing**: Set root divisions 3–4 inches deep and 2–3 feet apart in mounded rows.
- **Sunlight**: Choose a location with full sun to partial shade.
- **Soil**: Ensure the soil is moist, well-drained, and rich, with a pH of 5.8–6.2.

Remember to cut and remove an

## Adding Tools
Session memory to make it a real chat experience, allowing to use previous messages to be considered before making new outputs.

SerpApi for USDA Hardiness zone lookup. Split in 2, 1 tool to find source with bias for plantmaps.com (as this seems to be the website with most coverage and most accurate information). And 1 tool to convert content of the link in easliy readable text. Allowing the correct info to be read from the found source.

PDF creation for plannings, using FPDF for PDF generation. Agent uses input and chat history to create personalized planning. With download link as output.

In [77]:
# Session memory
# In-process, per-session message storage
_SESSION_STORES = defaultdict(InMemoryChatMessageHistory)

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    """Return/create a chat history bucket for this session."""
    return _SESSION_STORES[session_id]

# Helper to wrap Runnable (QA chain, Agent, etc.) with session memory
def with_session_memory(runnable, *, input_key: str = "question", history_key: str = "history"):
    """
    Wrap a Runnable so it remembers past turns per session.
    - input_key: the key in the invoke(...) dict that carries the user's new message
    - history_key: the key that the prompt/agent expects to receive past messages under
    """
    return RunnableWithMessageHistory(
        runnable,
        get_session_history,
        input_messages_key=input_key,
        history_messages_key=history_key,
    )


In [78]:
# USDA / World hardiness zone lookup (SerpAPI, Plantmaps-biased)
# Stable English results so titles/snippets are predictable
_serp = SerpAPIWrapper(params={"engine": "google", "hl": "en", "gl": "us", "num": 10})

def _first_plantmaps(organic_results):
    """Return first Plantmaps result url if present, else None."""
    for r in organic_results or []:
        url = (r.get("link") or "").strip()
        if "plantmaps.com" in url:
            return url
    return None

@tool("find_hardiness_source", return_direct=False)
def find_hardiness_source(location: str) -> str:
    """
    Return the best page to read the hardiness zone for a location.
    Prefers Plantmaps; falls back to the top Google result for 'hardiness zone <location>'.
    Output: JSON string with {location, best_source, note}.
    """
    if not location or not location.strip():
        return json.dumps({"error": "location required"})

    # 1) Plantmaps-biased query
    q_pm = f"site:plantmaps.com {location} hardiness zone"
    try:
        res_pm = _serp.results(q_pm)
    except Exception as e:
        return json.dumps({"location": location, "best_source": None,
                           "note": f"search error (plantmaps query): {e}"})

    best = _first_plantmaps(res_pm.get("organic_results"))
    if best:
        return json.dumps({"location": location, "best_source": best,
                           "note": "Plantmaps result chosen."})

    # 2) Fallback: generic query
    q_generic = f"hardiness zone {location}"
    try:
        res_generic = _serp.results(q_generic)
    except Exception as e:
        return json.dumps({"location": location, "best_source": None,
                           "note": f"search error (generic query): {e}"})

    # pick the very first organic result if available
    org = (res_generic.get("organic_results") or [])
    best = (org[0].get("link") if org else None)
    return json.dumps({"location": location, "best_source": best,
                       "note": "Generic result chosen." if best else "No results found."})


In [79]:
print(find_hardiness_source.invoke({"location": "Berlin, Germany"}))
print(find_hardiness_source.invoke({"location": "Madison, WI, USA"}))
print(find_hardiness_source.invoke({"location": "Groningen, Netherlands"}))


{"location": "Berlin, Germany", "best_source": "https://www.plantmaps.com/interactive-germany-plant-hardiness-zone-map-celsius.php", "note": "Plantmaps result chosen."}
{"location": "Madison, WI, USA", "best_source": "https://www.plantmaps.com/hardiness-zones-for-madison-wisconsin", "note": "Plantmaps result chosen."}
{"location": "Groningen, Netherlands", "best_source": "https://www.plantmaps.com/interactive-netherlands-plant-hardiness-zone-map-celsius.php", "note": "Plantmaps result chosen."}


In [80]:
# Read Plantmaps page and extract the zone (with 403 fallback)
# token like "Zone 8a" / "zone 8" etc.
_ZONE_RE = re.compile(r"\bZone\s*([0-9]{1,2}[ab]?)\b", re.I)

# build a set of fuzzy tokens from the user's location to match rows/lines
def _loc_tokens(location: str):
    s = location.lower()
    # split on commas and spaces, keep tokens with letters
    parts = [p.strip() for p in re.split(r"[,\s]+", s) if p.strip()]
    # drop very short noise like “us”, “the”
    parts = [p for p in parts if len(p) > 2]
    return set(parts)

def _pick_nearest_zone(lines: list[str], location: str) -> Optional[str]:
    tokens = _loc_tokens(location)
    best_idx, best_zone = None, None

    # 1) exact “City” line match first (common on Plantmaps country pages)
    for i, line in enumerate(lines):
        l = line.lower()
        # a row often looks like: "Berlin Zone 8a: -12.2°C to -9.4°C"
        if all(t in l for t in tokens):
            m_here = _ZONE_RE.search(line)
            if m_here:
                return m_here.group(1).lower()
            # if the zone is on the next line (sometimes), grab it
            if i + 1 < len(lines):
                m_next = _ZONE_RE.search(lines[i + 1])
                if m_next:
                    return m_next.group(1).lower()
            # otherwise remember index, we’ll expand search a bit
            if best_idx is None:
                best_idx = i

    # 2) if no direct hit, search a window around the first approximate hit
    if best_idx is not None:
        for j in range(max(0, best_idx - 3), min(len(lines), best_idx + 4)):
            m = _ZONE_RE.search(lines[j])
            if m:
                return m.group(1).lower()

    # 3) last resort: first zone anywhere on the page (not ideal, but better than nothing)
    for line in lines:
        m = _ZONE_RE.search(line)
        if m:
            return m.group(1).lower()

    return None

def _fetch_text_with_fallback(url: str) -> tuple[Optional[str], str]:
    """Fetch HTML or readable text from URL.
    Returns (text, method_used). Falls back to r.jina.ai on 403/other failures.
    """
    headers = {
        # very “real” browser headers help reduce 403s
        "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                       "AppleWebKit/537.36 (KHTML, like Gecko) "
                       "Chrome/126.0.0.0 Safari/537.36"),
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://www.google.com/",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
    }

    try:
        r = requests.get(url, headers=headers, timeout=12)
        if r.status_code == 200 and r.text:
            return r.text, "direct"
        # A tiny delay sometimes helps before retrying through the proxy
    except Exception:
        pass

    time.sleep(0.3)
    # Proxy reader that returns a plaintext rendering of the page
    proxy_url = f"https://r.jina.ai/http://{url.replace('https://', '').replace('http://', '')}"
    try:
        r2 = requests.get(proxy_url, headers={"User-Agent": headers["User-Agent"]}, timeout=12)
        if r2.status_code == 200 and r2.text:
            return r2.text, "r.jina.ai"
    except Exception:
        pass

    return None, "failed"

@tool("read_zone_from_url", return_direct=False)
def read_zone_from_url(url: str, location: Optional[str] = None) -> str:
    """
    Fetch a Plantmaps (or similar) page and extract a single hardiness zone.
    - Tries a normal fetch with realistic headers.
    - On 403/other failure, falls back to a plaintext proxy (r.jina.ai).
    - Uses the provided `location` to pick the right row on city lists.
    Returns a JSON string: {"zone": "8a", "source": url, "method": "...", "note": "..."}.
    """
    if not url:
        return json.dumps({"zone": None, "source": None, "method": None, "note": "No URL provided"})

    text, method = _fetch_text_with_fallback(url)
    if not text:
        return json.dumps({"zone": None, "source": url, "method": None, "note": "Fetch failed"})

    # Split to lines and normalize
    lines = [ln.strip() for ln in text.splitlines() if ln.strip()]
    zone = _pick_nearest_zone(lines, location or "")

    note = "Parsed from Plantmaps page via direct fetch." if method == "direct" \
        else "Parsed from Plantmaps page via r.jina.ai text proxy."

    return json.dumps({
        "zone": zone,
        "source": url,
        "note": note if zone else note + " No explicit zone token found."
    })


In [81]:
loc_test='Groningen'
src_json = find_hardiness_source.invoke({"location": loc_test})
print("SOURCE:", src_json)

src = json.loads(src_json)
page_json = read_zone_from_url.invoke({"url": src["best_source"], "location": loc_test})
print("PAGE READ:", page_json)


SOURCE: {"location": "Groningen", "best_source": "https://www.plantmaps.com/interactive-netherlands-plant-hardiness-zone-map-celsius.php", "note": "Plantmaps result chosen."}
PAGE READ: {"zone": "8a", "source": "https://www.plantmaps.com/interactive-netherlands-plant-hardiness-zone-map-celsius.php", "note": "Parsed from Plantmaps page via r.jina.ai text proxy."}


In [82]:
# PDF TOOL config.
# Safety net: force latin-1 safe text (replaces unsupported chars)
def latin1(s: str) -> str:
    return "" if s is None else str(s).encode("latin-1", "replace").decode("latin-1")

# Months in display order
MONTHS_ORDER = [
    "January","February","March","April","May","June",
    "July","August","September","October","November","December"
]

class PlannerPDF(FPDF):
    def header(self):
        self.set_font("Helvetica", "B", 16)
        # Title (single line, explicit width)
        self.set_x(self.l_margin)
        self.cell(self.w - self.l_margin - self.r_margin, 10,
                  getattr(self, "title", "Home Growing Planner"),
                  new_x="LMARGIN", new_y="NEXT")

        subtitle = getattr(self, "subtitle", "")
        if subtitle:
            self.set_x(self.l_margin)
            self.set_font("Helvetica", "", 11)
            self.set_text_color(80, 80, 80)
            # Wrap subtitle with explicit width
            self.multi_cell(self.w - self.l_margin - self.r_margin, 6, subtitle)
            self.set_text_color(0, 0, 0)

        self.ln(2)
        self.set_draw_color(220, 220, 220)
        self.line(self.l_margin, self.get_y(), self.w - self.r_margin, self.get_y())
        self.ln(4)

    def footer(self):
        self.set_y(-15)
        self.set_x(self.l_margin)
        self.set_font("Helvetica", "I", 9)
        self.set_text_color(120,120,120)
        # Footer: explicit width + centered
        self.cell(self.w - self.l_margin - self.r_margin, 10,
                  f"Generated {datetime.now().strftime('%Y-%m-%d')}  -  Page {self.page_no()}",
                  align="C")
        self.set_text_color(0,0,0)

def _order_months(keys: List[str]) -> List[str]:
    order = {m:i for i,m in enumerate(MONTHS_ORDER)}
    return sorted(keys, key=lambda k: order.get(k, 999))

def make_planner_pdf(
    path: str,
    *,
    location: str,
    zone: Optional[str],
    tasks_by_month: Dict[str, List[str]],
    crops: Optional[List[str]] = None,
    notes: str = ""
) -> Dict[str, object]:
    """
    Create a printable growing planner PDF.
    - location: free text (e.g., "Berlin, Germany")
    - zone: e.g., "8a" (can be None)
    - tasks_by_month: {"March": ["Start tomatoes indoors", ...], ...}
    - crops: optional ["tomato","onion",...]
    - notes: optional free text
    Returns: {"path": path, "pages": int, "bytes": int}
    """
    pdf = PlannerPDF(orientation="P", unit="mm", format="A4")
    pdf.set_auto_page_break(auto=True, margin=15)

    subtitle_bits = [latin1(f"Location: {location}")]
    if zone:
        subtitle_bits.append(latin1(f"Zone: {zone}"))
    if crops:
        subtitle_bits.append(latin1("Crops: " + ", ".join(crops)))
    pdf.title = "Home Growing Planner"
    pdf.subtitle = "  -  ".join(subtitle_bits)

    pdf.add_page()

    # Overview (only if notes are provided)
    if notes:
        pdf.set_font("Helvetica", "B", 13)
        pdf.set_x(pdf.l_margin)
        pdf.cell(pdf.w - pdf.l_margin - pdf.r_margin, 8, "Overview",
                 new_x="LMARGIN", new_y="NEXT")
        pdf.set_font("Helvetica", "", 12)
        pdf.multi_cell(pdf.w - pdf.l_margin - pdf.r_margin, 6, latin1(notes))
        pdf.ln(2)

    # Month-by-month heading
    pdf.set_font("Helvetica", "B", 13)
    pdf.set_x(pdf.l_margin)
    pdf.cell(pdf.w - pdf.l_margin - pdf.r_margin, 8, "Month-by-Month",
             new_x="LMARGIN", new_y="NEXT")
    pdf.set_font("Helvetica", "", 11)

    # Define months from keys (ordered)
    months = _order_months(list(tasks_by_month.keys()))

    # Month sections
    for m in months:
        pdf.set_font("Helvetica", "B", 12)
        pdf.set_x(pdf.l_margin)
        pdf.multi_cell(pdf.w - pdf.l_margin - pdf.r_margin, 7, latin1(m))

        pdf.set_font("Helvetica", "", 11)
        rows = tasks_by_month.get(m, [])
        if not rows:
            pdf.set_text_color(120,120,120)
            pdf.set_x(pdf.l_margin)
            pdf.cell(pdf.w - pdf.l_margin - pdf.r_margin, 6, "No specific tasks.",
                     new_x="LMARGIN", new_y="NEXT")
            pdf.set_text_color(0,0,0)
        else:
            for task in rows:
                pdf.set_x(pdf.l_margin)
                pdf.multi_cell(pdf.w - pdf.l_margin - pdf.r_margin, 6, latin1(f"- {task}"))
        pdf.ln(1)

    # Save to disk so the agent/UI can return a downloadable path
    outdir = os.path.dirname(path)
    if outdir:
        os.makedirs(outdir, exist_ok=True)
    pdf.output(path)
    return {"path": path, "pages": pdf.page_no(), "bytes": os.path.getsize(path)}

In [83]:
# how to call the tool, will go in to the agent but now just here to test.
@tool("make_planner_pdf", return_direct=False)
def make_planner_pdf_tool(payload_json: str) -> str:
    """
    Create a printable growing planner PDF.
    Args (JSON string):
      location: str
      zone: str | null
      tasks_by_month: dict[str, list[str]]
      crops: list[str] | null
      notes: str | ""
      out_path: optional str (defaults to './planner_<timestamp>.pdf')
    Returns JSON: {"path": ".../planner_xxx.pdf", "pages": N, "bytes": M}
    """
    data = json.loads(payload_json)
    out_path = data.get("out_path") or f"./planner_{int(time.time())}.pdf"
    result = make_planner_pdf(
        path=out_path,
        location=data["location"],
        zone=data.get("zone"),
        tasks_by_month=data["tasks_by_month"],
        crops=data.get("crops") or None,
        notes=data.get("notes",""),
    )
    return json.dumps(result)


In [84]:
# Testing pdf creation tool, manually inputing information. Just to check what the outcome looks like.
example = {
    "location": "Berlin, Germany",
    "zone": "8a",
    "tasks_by_month": {
        "March": ["Start tomato seeds indoors", "Sow peas outdoors"],
        "April": ["Harden off seedlings", "Plant onions sets"],
        "May": ["Transplant tomatoes after last frost", "Direct sow beans"]
    },
    "crops": ["tomato","pea","onion","bean"],
    "notes": "Planner generated from your preferences and local zone data."
}
print(make_planner_pdf_tool.run(json.dumps(example)))


{"path": "./planner_1758264040.pdf", "pages": 1, "bytes": 1696}


## New QA with updated prompt and tools
This qa chain now includes the tools that we have made above, which should result in a better chat experiece. And have a few extra functions besides just chatting, since we have added the zone lookup and pdf planning. It uses the selfquery retriever by invoking rag_chain

In [85]:
# QA Agent with tools (Self-Query RAG for Q&A + Zone lookup + PDF)
# Wrap existing RAG chain as a tool so the agent uses it for answers.
#    NOTE: rag_chain must already be defined as in earlier cell
#    (it uses self_query_retriever under the hood).
@tool("growguide_answer", return_direct=False)
def growguide_answer(question: str) -> str:
    """
    Answer the user's question using the Grow Guide RAG QA chain.
    (This chain already uses the self-query retriever.)
    Returns a concise, step-by-step answer.
    """
    return rag_chain.invoke(question)

# 1) Expose the retriever as a raw-search tool for planner building.
@tool("search_growguide", return_direct=False)
def search_growguide(query: str) -> str:
    """
    Retrieve up to 6 relevant chunks from the Grow Guide using the Self-Query retriever.
    Returns a JSON list of items: [{"content": "...", "meta": {...}}, ...]
    Use this to extract month-specific tasks, crop tips, etc., e.g., when building a PDF planner.
    """
    hits = self_query_retriever.get_relevant_documents(query) or []
    out = [{"content": h.page_content, "meta": h.metadata} for h in hits[:6]]
    return json.dumps(out)

# 2) Agent prompt: route to the right tool
agent_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a helpful home-growing assistant.\n"
     "TOOLS:\n"
     "• growguide_answer — Use for most Q&A. It runs the RAG QA chain (self-query retriever) and returns a ready answer.\n"
     "• search_growguide — Use when you need raw snippets to assemble tasks_by_month for a PDF planner.\n"
     "• find_hardiness_source + read_zone_from_url — Use to determine a location's hardiness zone.\n\n"
     "Guidance:\n"
     "1) If the user asks for their hardiness zone: call 'find_hardiness_source' with the location, then "
     "'read_zone_from_url' with that URL (and the location) to extract the zone; remember it in this chat.\n"
     "2) For normal growing questions: call 'growguide_answer' with the user's question (include any known zone/crop/months).\n"
     "3) If a full planner/PDF is requested: call 'search_growguide' to fetch snippets, build tasks_by_month (month -> [task,...]), "
     "then call 'make_planner_pdf' with location, zone (if known), tasks_by_month, optional crops, and notes. Use ASCII/latin-1 text.\n"
     "4) Keep answers focused and brief. Directly answer the user's question **only**. Prefer 1–4 sentences or a short bullet list. "
     "Avoid extra detail unless asked; instead, offer a follow-up like: "
     "\"I can share more detail (e.g., step-by-step or month-by-month tasks) if you’d like.\"\n"
     "5) Stay in scope: home gardening of fruits/vegetables, hardiness climate zones, and printable planning PDFs. "
     "If a request is outside this scope (e.g., gaming, finance, unrelated tech), reply briefly: "
     "\"Sorry, that’s outside my expertise. I can help with home fruit & vegetable growing, climate zones, and personalized planning.\""
     ),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),
])

# 3) LLM + tools
llm_tools = [
    growguide_answer,        # <- preferred for Q&A (uses self-query retriever via rag_chain)
    search_growguide,        # <- raw chunks for planner building (also uses self-query retriever)
    find_hardiness_source,   # SerpAPI step 1 (best source URL as JSON)
    read_zone_from_url,      # SerpAPI step 2 (parse page to get zone)
    make_planner_pdf_tool,   # PDF generator tool
]

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

agent = create_openai_tools_agent(llm, llm_tools, agent_prompt)
agent_exec = AgentExecutor(
    agent=agent,
    tools=llm_tools,
    verbose=True,
    max_iterations=8,
    handle_parsing_errors=True,
).with_config({"run_name": "growguide_tools_agent", "tags": ["agent","tools","rag"]})

print("QA Tools Agent ready.")

# 4) Session memory wrapper that was already built
qa_agent = with_session_memory(agent_exec, input_key="input", history_key="chat_history")
print("QA Tools Agent with session memory ready.")

QA Tools Agent ready.
QA Tools Agent with session memory ready.


## Testing Agent
We kept verbose on True in the agent to see already in the notebook what is happening. Will be turned off in the deployment.

In [86]:
# --- Example calls (uncomment to try) ---
session = {"configurable": {"session_id": "u5"}}

# 1) Zone lookup
qa_agent.invoke({"input": "What's the hardiness zone for Barcelona?"}, config=session)



> Entering new growguide_tools_agent chain...

Invoking: `find_hardiness_source` with `{'location': 'Barcelona'}`


{"location": "Barcelona", "best_source": "https://www.plantmaps.com/interactive-spain-plant-hardiness-zone-map-celsius.php", "note": "Plantmaps result chosen."}
Invoking: `read_zone_from_url` with `{'url': 'https://www.plantmaps.com/interactive-spain-plant-hardiness-zone-map-celsius.php', 'location': 'Barcelona'}`


{"zone": "10a", "source": "https://www.plantmaps.com/interactive-spain-plant-hardiness-zone-map-celsius.php", "note": "Parsed from Plantmaps page via r.jina.ai text proxy."}The hardiness zone for Barcelona is 10a.

> Finished chain.


{'input': "What's the hardiness zone for Barcelona?",
 'chat_history': [],
 'output': 'The hardiness zone for Barcelona is 10a.'}

In [87]:
# Keep asking questions but with less context to see if it will use the memory
qa_agent.invoke({"input": "Can i grow carrots in my location? When should i plant them?"}, config=session)



> Entering new growguide_tools_agent chain...

Invoking: `growguide_answer` with `{'question': 'Can I grow carrots in zone 10a?'}`


Yes, you can grow carrots in zone 10a. Carrots thrive in cooler temperatures, so it's best to plant them in the fall or early spring when temperatures are milder. Make sure to provide well-drained soil and consistent moisture for optimal growth. If you need more specific planting or care tips, let me know!
Invoking: `growguide_answer` with `{'question': 'When should I plant carrots in zone 10a?'}`


In zone 10a, you can plant carrots in the fall or winter months, typically from September to February. This timing allows them to grow in cooler temperatures, which is ideal for their development. If you'd like, I can provide a month-by-month plan for growing carrots!Yes, you can grow carrots in zone 10a. It's best to plant them in the fall or early spring, typically from September to February, when temperatures are milder. Ensure you provide well-drained so

{'input': 'Can i grow carrots in my location? When should i plant them?',
 'chat_history': [HumanMessage(content="What's the hardiness zone for Barcelona?", additional_kwargs={}, response_metadata={}),
  AIMessage(content='The hardiness zone for Barcelona is 10a.', additional_kwargs={}, response_metadata={})],
 'output': "Yes, you can grow carrots in zone 10a. It's best to plant them in the fall or early spring, typically from September to February, when temperatures are milder. Ensure you provide well-drained soil and consistent moisture for optimal growth. If you want a detailed month-by-month plan for growing carrots, just let me know!"}

In [88]:
qa_agent.invoke({"input": "Could you make me a full planning for what I should do between April and August?"}, config=session)



> Entering new growguide_tools_agent chain...

Invoking: `search_growguide` with `{'query': 'carrots planting and care tasks April May June July August'}`


[{"content": "Beets: 45\u201360 days (Plant successions at recommended intervals for species.)\nBroccoli: 70\u2013100 days\nBrussels Sprouts: 100\u2013110 days\nCabbage: 50\u201360 days (Plant successions at recommended intervals for species.)\nCarrots: 50\u201375 days (Plant successions at recommended intervals for species.)\nCauliflower: 70\u2013120 days\nCelery: 100\u2013120 days\nChinese Cabbage: 50\u201385 days (Plant successions at recommended intervals for species.)\nCollards: 60\u201390 days (Plant successions at recommended intervals for species.)\nCucumbers: 55\u201365 days\nEggplants: 100\u2013140 days\nEndive: 85\u2013100 days (Plant successions at recommended intervals for species.)\nGourds: 85\u2013100 days\nKale: 55\u201375 days (Plant successions at recommended intervals for species.)\nLettuce, Head: 65\u201390 da

{'input': 'Could you make me a full planning for what I should do between April and August?',
 'chat_history': [HumanMessage(content="What's the hardiness zone for Barcelona?", additional_kwargs={}, response_metadata={}),
  AIMessage(content='The hardiness zone for Barcelona is 10a.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Can i grow carrots in my location? When should i plant them?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Yes, you can grow carrots in zone 10a. It's best to plant them in the fall or early spring, typically from September to February, when temperatures are milder. Ensure you provide well-drained soil and consistent moisture for optimal growth. If you want a detailed month-by-month plan for growing carrots, just let me know!", additional_kwargs={}, response_metadata={})],
 'output': "I've created a full planting planner for growing carrots in Barcelona from April to August. You can download it using the link below:

In [89]:
qa_agent.invoke({"input": "Could you actually make the planning for all crops that would work well for my zone, between April and August?"}, config=session)



> Entering new growguide_tools_agent chain...

Invoking: `search_growguide` with `{'query': 'crops for zone 10a April to August'}`


[]It seems I couldn't find specific crops for zone 10a between April and August. However, common crops that typically thrive in this zone during that period include:

- **Tomatoes**: Plant in April for summer harvest.
- **Peppers**: Start in April for a summer yield.
- **Eggplants**: Also best planted in April.
- **Cucumbers**: Sow seeds in April for summer growth.
- **Beans**: Plant in May for a summer harvest.
- **Squash**: Sow in May for summer production.
- **Carrots**: Continue planting in April for a late summer harvest.

If you'd like, I can create a detailed month-by-month planner for these crops. Let me know!

> Finished chain.


{'input': 'Could you actually make the planning for all crops that would work well for my zone, between April and August?',
 'chat_history': [HumanMessage(content="What's the hardiness zone for Barcelona?", additional_kwargs={}, response_metadata={}),
  AIMessage(content='The hardiness zone for Barcelona is 10a.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Can i grow carrots in my location? When should i plant them?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Yes, you can grow carrots in zone 10a. It's best to plant them in the fall or early spring, typically from September to February, when temperatures are milder. Ensure you provide well-drained soil and consistent moisture for optimal growth. If you want a detailed month-by-month plan for growing carrots, just let me know!", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Could you make me a full planning for what I should do between April and August?', additiona

In [90]:
qa_agent.invoke({"input": "Yes, could you generate the PDF for me with a planning for the crops that you found?"}, config=session)



> Entering new growguide_tools_agent chain...

Invoking: `search_growguide` with `{'query': 'April to August planting guide for zone 10a crops'}`


[]
Invoking: `growguide_answer` with `{'question': 'What crops can be grown in hardiness zone 10a between April and August?'}`


In hardiness zone 10a, you can grow a variety of crops between April and August. Some suitable options include:

- Tomatoes
- Peppers
- Eggplants
- Cucumbers
- Squash
- Beans
- Melons
- Corn

These crops thrive in the warm temperatures typical of this zone during those months. I can share step-by-step details or a month-by-month plan if you’d like!
Invoking: `growguide_answer` with `{'question': 'What are the month-by-month tasks for growing tomatoes, peppers, eggplants, cucumbers, beans, squash, and carrots in hardiness zone 10a from April to August?'}`


Here are the month-by-month tasks for growing tomatoes, peppers, eggplants, cucumbers, beans, squash, and carrots in hardiness zone 10a from April to August:


{'input': 'Yes, could you generate the PDF for me with a planning for the crops that you found?',
 'chat_history': [HumanMessage(content="What's the hardiness zone for Barcelona?", additional_kwargs={}, response_metadata={}),
  AIMessage(content='The hardiness zone for Barcelona is 10a.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Can i grow carrots in my location? When should i plant them?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Yes, you can grow carrots in zone 10a. It's best to plant them in the fall or early spring, typically from September to February, when temperatures are milder. Ensure you provide well-drained soil and consistent moisture for optimal growth. If you want a detailed month-by-month plan for growing carrots, just let me know!", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Could you make me a full planning for what I should do between April and August?', additional_kwargs={}, response_meta

## Evaluation

In [91]:
# Useing the same `agent` and `llm_tools`.
# But this executor does NOT include memory and returns tool traces.
agent_exec_eval = AgentExecutor(
    agent=agent,
    tools=llm_tools,
    verbose=False,                 # quiet for evaluation
    max_iterations=8,
    handle_parsing_errors=True,
    return_intermediate_steps=True # tool traces
).with_config({"run_name": "growguide_tools_agent_eval", "tags": ["agent","tools","eval"]})


In [92]:
# Functions for the evaluation
def _safe_json_loads(s: str):
    try:
        return json.loads(s)
    except Exception:
        try:
            # light repair for curly/odd quotes
            return json.loads(s.replace("“","\"").replace("”","\"").replace("'", "\""))
        except Exception:
            return None

def _collect_context_from_steps(steps):
    """
    Build plain-text CONTEXT that mirrors what the agent saw.
    - For `search_growguide`: parse the observation (JSON/list) and concatenate each item's `content`.
    - For other tools (zone lookup, etc.): include a short readable snippet so the judge can see evidence.
    """
    ctx_bits = []

    for action, observation in steps:
        tool_name = getattr(action, "tool", "") or getattr(action, "tool_name", "")
        obs = observation

        # Normalize observation into a Python object if it's JSON text
        data = _safe_json_loads(obs) if isinstance(obs, str) else obs

        if tool_name == "search_growguide":
            items = data if data is not None else obs
            # tolerate both [{"content":...},...] and {"items":[...]}
            if isinstance(items, dict) and "items" in items:
                items = items["items"]

            contents = []
            if isinstance(items, list):
                for it in items:
                    if isinstance(it, dict):
                        c = it.get("content") or it.get("page_content") or it.get("text") or ""
                    else:
                        c = str(it)
                    if c:
                        contents.append(c.strip())
            elif isinstance(items, dict):
                c = items.get("content") or items.get("page_content") or items.get("text") or ""
                if c:
                    contents.append(c.strip())

            if contents:
                ctx_bits.append("\n\n".join(contents))

        else:
            # keep short, readable evidence for non-retrieval tools
            snippet = obs
            if not isinstance(snippet, str):
                try:
                    snippet = json.dumps(snippet, ensure_ascii=False)
                except Exception:
                    snippet = str(snippet)
            ctx_bits.append(snippet.strip())

    # Join and cap to keep the judge prompt reasonable
    ctx = "\n\n".join([c for c in ctx_bits if c])
    MAX_CHARS = 3000
    if len(ctx) > MAX_CHARS:
        ctx = ctx[:MAX_CHARS] + " …[truncated]"
    return ctx

def run_agent_once(question: str) -> dict:
    """Run the eval executor once and return {'answer': ..., 'context': ...}."""
    # supply empty chat_history because eval agent has no memory wrapper
    res = agent_exec_eval.invoke({"input": question, "chat_history": []})

    answer = (
        res.get("output")
        or res.get("final_output")
        or (res if isinstance(res, str) else str(res))
    )

    steps = res.get("intermediate_steps", [])
    context = _collect_context_from_steps(steps)

    return {"answer": answer, "context": context}


In [93]:
# Judge prompt with 1–10 scale 
judge_prompt = ChatPromptTemplate.from_template(
    """You are an impartial evaluator for a gardening QA system.

Score the assistant's answer on five dimensions from 1 to 10 (integers):
- correctness: factual accuracy
- groundedness: supported by the provided CONTEXT
- relevance: addresses the question
- completeness: covers the key steps/details expected
- conciseness: clear and not verbose

Rules for groundedness:
- If a claim is not supported by the CONTEXT (quoted below), deduct points.
- If CONTEXT is empty or does not contain supporting evidence, set groundedness to 1.
- You may not assume external knowledge beyond the CONTEXT for groundedness.

Return a single JSON object with exactly these keys:
correctness, groundedness, relevance, completeness, conciseness, rationale

Question:
{question}

Assistant answer:
{answer}

CONTEXT (the only ground truth to use):
{context}
"""
)

llm_judge = ChatOpenAI(model="gpt-4o-mini", temperature=0)
judge_chain = judge_prompt | llm_judge | StrOutputParser()

def _coerce_1_10(x):
    try:
        v = int(round(float(x)))
        return min(10, max(1, v))
    except Exception:
        return 1

def _parse_judge_json(raw: str) -> dict:
    # extract the first JSON object if the model adds extra text
    m = re.search(r"\{.*\}", raw, flags=re.S)
    blob = m.group(0) if m else "{}"
    try:
        data = json.loads(blob)
    except Exception:
        try:
            data = json.loads(blob.replace("“","\"").replace("”","\""))
        except Exception:
            data = {}

    return {
        "correctness":  _coerce_1_10(data.get("correctness", 1)),
        "groundedness": _coerce_1_10(data.get("groundedness", 1)),
        "relevance":    _coerce_1_10(data.get("relevance", 1)),
        "completeness": _coerce_1_10(data.get("completeness", 1)),
        "conciseness":  _coerce_1_10(data.get("conciseness", 1)),
        "rationale":    (data.get("rationale") or raw).strip(),
    }

def judge_sample(question: str, answer: str, context: str) -> dict:
    raw = judge_chain.invoke({"question": question, "answer": answer, "context": context})
    return _parse_judge_json(raw)


In [96]:
# Evaluation question set
EVAL_QUESTIONS = [
    "Best companion plants for peppers?",
    "How long does it take untill harvest for carrots?",
    "We are a family of 3, how many Leeks should we grow?",
    "What are the best soil conditions for growing onions?",
    "How long do pumpkin seeds take to germinate?",
    "Can you tell 4 or 5 easy maintance crops to grow?",
    "What is the best zone for growing chili peppers?",
    "Any pests or insect i should be aware of when growing tomatoes?",
    "When do I harvest garlic?",
    "What to be carefull of when growing rhubarb?"
]

REPEATS = 3  # run each question 3x to check consistency

def _consistency_1_10(texts):
    """Average pairwise similarity (0..1) -> scaled to 1..10."""
    if len(texts) < 2:
        return 10
    ratios = []
    for i in range(len(texts)):
        for j in range(i+1, len(texts)):
            ratios.append(SequenceMatcher(None, texts[i], texts[j]).ratio())
    avg = sum(ratios)/len(ratios) if ratios else 1.0
    # map 0..1 to 1..10 (so 0 -> 1, 1 -> 10)
    return max(1, min(10, int(round(avg*9 + 1))))

all_runs = []
for q in EVAL_QUESTIONS:
    runs = []
    for _ in range(REPEATS):
        out = run_agent_once(q)
        score = judge_sample(q, out["answer"], out["context"])
        runs.append({
            "answer": out["answer"],
            "context_len": len(out["context"]),
            **score
        })

    consistency = _consistency_1_10([r["answer"] for r in runs])

    agg = {
        "question": q,
        "n_runs": REPEATS,
        "avg_correctness":  round(sum(r["correctness"]  for r in runs)/REPEATS, 2),
        "avg_groundedness": round(sum(r["groundedness"] for r in runs)/REPEATS, 2),
        "avg_relevance":    round(sum(r["relevance"]    for r in runs)/REPEATS, 2),
        "avg_completeness": round(sum(r["completeness"] for r in runs)/REPEATS, 2),
        "avg_conciseness":  round(sum(r["conciseness"]  for r in runs)/REPEATS, 2),
        "avg_context_len":  round(sum(r["context_len"]  for r in runs)/REPEATS, 1),
        "consistency_1_10": consistency,
        "runs": runs,
    }
    all_runs.append(agg)

# Overall aggregates
overall = {
    "n_questions": len(all_runs),
    "correctness_mean":  round(sum(a["avg_correctness"]  for a in all_runs)/len(all_runs), 2),
    "groundedness_mean": round(sum(a["avg_groundedness"] for a in all_runs)/len(all_runs), 2),
    "relevance_mean":    round(sum(a["avg_relevance"]    for a in all_runs)/len(all_runs), 2),
    "completeness_mean": round(sum(a["avg_completeness"] for a in all_runs)/len(all_runs), 2),
    "conciseness_mean":  round(sum(a["avg_conciseness"]  for a in all_runs)/len(all_runs), 2),
    "consistency_mean":  round(sum(a["consistency_1_10"] for a in all_runs)/len(all_runs), 2),
}
overall

{'n_questions': 10,
 'correctness_mean': 9.73,
 'groundedness_mean': 8.8,
 'relevance_mean': 9.87,
 'completeness_mean': 8.53,
 'conciseness_mean': 8.87,
 'consistency_mean': 8.1}

In [97]:
# Shows answers and scores
for qres in all_runs:
    print("\n=== Q:", qres["question"], "===")
    for i, r in enumerate(qres["runs"], 1):
        print(f"\nRun {i}:")
        print("Answer:\n", r["answer"])
        print("Scores:",
              {k: r[k] for k in ("correctness","groundedness","relevance","completeness","conciseness")})




=== Q: Best companion plants for peppers? ===

Run 1:
Answer:
 The best companion plants for peppers include:

- Beets
- Garlic
- Onions
- Parsnips

These plants can enhance growth and help deter pests. If you need more details on growing these companions or their specific benefits, just let me know!
Scores: {'correctness': 10, 'groundedness': 10, 'relevance': 10, 'completeness': 8, 'conciseness': 9}

Run 2:
Answer:
 The best companion plants for peppers include:

- Beets
- Garlic
- Onions
- Parsnips

These plants can enhance growth and help deter pests. If you need more details on growing these companions or their specific benefits, just let me know!
Scores: {'correctness': 10, 'groundedness': 10, 'relevance': 10, 'completeness': 8, 'conciseness': 9}

Run 3:
Answer:
 The best companion plants for peppers include:

- Beets
- Garlic
- Onions
- Parsnips

These plants can enhance growth and help deter pests. If you need more details or specific growing tips, feel free to ask!
Scores: {'c